In [2]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import torch
from scipy.spatial.transform import Rotation
import laspy
import pandas as pd

# 1. NaviEdit -> LAZ -> Point cloud

In [49]:
def normalize_pc(points):
	centroid = np.mean(points, axis=0)
	points -= centroid
	furthest_distance = np.max(np.sqrt(np.sum(abs(points)**2,axis=-1)))
	points /= furthest_distance

	return points

In [77]:
las = laspy.read('/home/li/Downloads/EM2040-0030-L03S01-20220119-193553.las')
points = np.vstack((las.X, las.Y, las.Z)).transpose()
points = points * las.header.scale + las.header.offset
print(points.shape)
print(points[0])

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
downpcd = pcd.voxel_down_sample(voxel_size=2)
downpcd.points
#o3d.visualization.draw_geometries([downpcd])

aug_noise = .1
rot_factor = 1

down_points = np.asarray(downpcd.points)
print(down_points.shape)
src = np.random.permutation(down_points[:10000, :])[:2000, :]
src += (np.random.rand(*src.shape) - 0.5) * aug_noise
src = normalize_pc(src)
print(src.shape)
torch.save(src, "assets/test.pth")

# rotate the point cloud
trg = np.random.permutation(down_points[7000:18000, :])[:2000, :]
euler_ab=np.random.rand(3)*np.pi*2/rot_factor # anglez, angley, anglex
rot_ab= Rotation.from_euler('zyx', euler_ab).as_matrix()
trg += (np.random.rand(*trg.shape) - 0.5) * aug_noise
trg =np.matmul(rot_ab,trg.T).T
trg = normalize_pc(trg)
torch.save(trg, "assets/test1.pth")

(914502, 3)
[ 4.60723066e+05  1.77812617e+06 -9.04019980e+02]
(155618, 3)
(2000, 3)


In [64]:
#pcd = o3d.io.read_point_cloud("assets/cloud_bin_21.pth")
data = torch.load("assets/test.pth").astype(np.float32)
data1 = torch.load("assets/test1.pth").astype(np.float32)
print(data.shape, data1.shape)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(data)

pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(data1)

o3d.visualization.draw_geometries([pcd, pcd1])

(2000, 3) (2000, 3)


In [67]:
np.array(pcd.points)

array([[ 0.33896023, -0.43553659,  0.04193757],
       [ 0.00954625,  0.34624669, -0.0106827 ],
       [ 0.31984845,  0.06498687,  0.02973026],
       ...,
       [-0.17993498, -0.28495371, -0.03518974],
       [-0.53547716, -0.78250396, -0.02612695],
       [-0.39036116, -0.24462132, -0.03694123]])

# 2. NaviEdit -> XYZ data beam -> Point cloud
- Why?
    1. (Important) .las exports points in (num_points, 3) and it's a bit hard to tell which points are from the same beam, which makes it a bit difficult to separete based on time + space
    2. (Not so important) the timestamp from the .las export seems to be off (~year 1990)...

In [3]:
def process_xyz_pings_file(filepath):
    """Read a .xyz file containing info about each beam hits and return a processed dataframe
    where each row contains info for 1 MBES swath"""
    naviedit_pings = pd.read_csv(filepath, delimiter='\t', header=0)

    # Aggregate xyz hits of each beam into a hits array
    naviedit_pings['hit'] = naviedit_pings[['X', 'Y',
                                            'Z']].apply(lambda row: list(row),
                                                        axis=1)

    # Parse string datetime values and produce a datetime object
    naviedit_pings['time_string'] = naviedit_pings[[
        'yyyy', 'mmddhhmm', 'ss.ss'
    ]].apply(lambda row: ''.join(row.values.astype(str)), axis=1)
    naviedit_pings['time_string'] = pd.to_datetime(
        naviedit_pings['time_string'], format='%Y.%m%d%H%M.0%S.%f')
    naviedit_pings['time_stamp'] = naviedit_pings['time_string'].map(
        pd.Timestamp.timestamp)

    # Group the beams by Ping No -> aggregate hits from the same ping into a list
    beams = naviedit_pings.groupby('Ping No').agg({'hit': list})

    # Get unique pings
    ping_info = naviedit_pings[[
        'time_string', 'time_stamp', 'Ping No', 'Tide', 'Heading', 'Heave',
        'Pitch', 'Roll'
    ]].drop_duplicates().set_index('Ping No')

    # Final results with 1 row = 1 MBES swath
    mbes_ping_result = ping_info.join(beams)
    return mbes_ping_result


In [10]:
df = process_xyz_pings_file('/home/li/Downloads/EM2040-0030-L03S01-20220119-193553.xyz')
df.head()

,time_string,time_stamp,Tide,Heading,Heave,Pitch,Roll,hit
Ping No,,,,,,,,
39909,2022-01-19 19:35:53.060,1.642621e+09,0.0,271.83,0.0,11.702,-1.58,"[[460722.21, 1778135.04, 904.33], [460722.14, ..."
39910,2022-01-19 19:35:53.410,1.642621e+09,0.0,271.80,0.0,12.408,-1.61,"[[460720.75, 1778144.44, 904.97], [460720.69, ..."
39911,2022-01-19 19:35:53.780,1.642621e+09,0.0,271.78,0.0,13.165,-1.64,"[[460720.02, 1778146.45, 904.93], [460719.95, ..."
39912,2022-01-19 19:35:54.140,1.642621e+09,0.0,271.75,0.0,13.942,-1.69,"[[460719.88, 1778143.95, 904.88], [460719.79, ..."
39913,2022-01-19 19:35:54.500,1.642621e+09,0.0,271.72,0.0,14.718,-1.71,"[[460718.38, 1778150.95, 904.99], [460718.21, ..."


In [14]:
naviedit_pings = pd.read_csv('/home/li/Downloads/EM2040-0030-L03S01-20220119-193553.xyz', delimiter='\t', header=0)
naviedit_pings.head()

,yyyy,mmddhhmm,ss.ss,Ping No,Beam No,X,Y,Z,Tide,Heading,Heave,Pitch,Roll
0,2022,1191935,53.06,39909,24,460722.21,1778135.04,904.33,0.0,271.83,0.0,11.702,-1.58
1,2022,1191935,53.06,39909,25,460722.14,1778135.93,904.26,0.0,271.83,0.0,11.702,-1.58
2,2022,1191935,53.06,39909,26,460722.07,1778136.49,904.41,0.0,271.83,0.0,11.702,-1.58
3,2022,1191935,53.06,39909,27,460721.99,1778137.07,904.58,0.0,271.83,0.0,11.702,-1.58
4,2022,1191935,53.06,39909,28,460721.93,1778137.96,904.46,0.0,271.83,0.0,11.702,-1.58


In [38]:
nbr_pings = naviedit_pings['Ping No'].unique().shape[0]
max_nbr_beams = 400
print(f'nbr pings: {nbr_pings}, max nbr beams: {max_nbr_beams}')
naviedit_pings['ping_id'] = naviedit_pings['Ping No'] - naviedit_pings['Ping No'].min()

nbr pings: 2460, max nbr beams: 400


In [78]:
# iterate through naviedit_pings and populate pings_array
pings_array = np.zeros((nbr_pings, max_nbr_beams, 3))
for i, row in naviedit_pings.iterrows():
    ping_id = row['ping_id'].astype(int)
    beam_id = row['Beam No'].astype(int)
    pings_array[ping_id, beam_id, :] = row[['X', 'Y', 'Z']]

In [120]:
section = pings_array[:500, :200, :].reshape(-1, 3)
tmp2 = pings_array[:500, 200:, :].reshape(-1, 3)
mask = ~np.all(section == 0, axis=1)
mask2 = ~np.all(tmp2 == 0, axis=1)
print(f'% beams with data: {np.count_nonzero(mask)/section.shape[0]*100:.2f}%')
tmp_pcd = o3d.geometry.PointCloud()
tmp_pcd.points = o3d.utility.Vector3dVector(section[mask, :])
tmp2_pcd = o3d.geometry.PointCloud()
tmp2_pcd.points = o3d.utility.Vector3dVector(tmp2[mask2, :])
o3d.visualization.draw_geometries([tmp_pcd, tmp2_pcd])

% beams with data: 84.46%


In [121]:
def naviedit_xyz_to_npy(filepath, max_nbr_beams=400):
    """Given a .xyz file exported by NaviEdit with xyz info for each beam hit,
    return a numpy array of shape (nbr_pings, max_nbr_beams, 3) where 
    array[i, j, :] = [x, y, z] of the jth beam hit in the ith ping. Beam with no
    hit will be filled with 0s."""

    pings = pd.read_csv(filepath, delimiter='\t', header=0)
    nbr_pings = pings['Ping No'].unique().shape[0]
    print(f'nbr pings: {nbr_pings}, max nbr beams: {max_nbr_beams}')
    pings['ping_id'] = pings['Ping No'] - pings['Ping No'].min()

    # iterate through naviedit_pings and populate pings_array
    pings_array = np.zeros((nbr_pings, max_nbr_beams, 3))
    for i, row in pings.iterrows():
        ping_id = row['ping_id'].astype(int)
        beam_id = row['Beam No'].astype(int)
        pings_array[ping_id, beam_id, :] = row[['X', 'Y', 'Z']]

    return pings_array

def plot_pcl_section(pings_array, start_ping=0, end_ping=500, start_beam=0, end_beam=200):
    """Plot a section of the pings_array"""
    section = pings_array[start_ping:end_ping, start_beam:end_beam, :].reshape(-1, 3)
    nonzero_mask = ~np.all(section == 0, axis=1)
    print(f'ping range: {start_ping} - {end_ping}, beam range: {start_beam} - {end_beam}')
    print(f'% beams with data: {np.count_nonzero(nonzero_mask)/section.shape[0]*100:.2f}%')
    section_pcd = o3d.geometry.PointCloud()
    section_pcd.points = o3d.utility.Vector3dVector(section[nonzero_mask, :])
    o3d.visualization.draw_geometries([section_pcd])

In [125]:
plot_pcl_section(pings_array, start_ping=1000, end_ping=1500, start_beam=0, end_beam=100)

ping range: 100 - 2000, beam range: 0 - 400
% beams with data: 89.01%
